# TFRecords files

* [`tf.slim site`](https://github.com/tensorflow/models/tree/master/research/slim)
* 위의 링크에 나온 방법으로 TensorFlow에서 제공하는 `flower` 데이터 셋을 download 받고
* 그 후 `TFRecords` format의 데이터로 만들어보자.

## Downloading and converting to TFRecord format

```shell
$ DATA_DIR=/tmp/data/flowers
$ python download_and_convert_data.py \
    --dataset_name=flowers \
    --dataset_dir="${DATA_DIR}"
```
* script 파일을 그대로 이용하여도 된다.
* `flower` 데이터 셋을 download 한 이후 `TFRecords` format으로 변환

In [2]:
import sys
sys.path.insert(0, 'D:/PythonLab/CS20/models-master/research/slim/')

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
import random
import tarfile

from six.moves import urllib

import tensorflow as tf

from datasets import dataset_utils

In [4]:
# The URL where the Flowers data can be downloaded.
_DATA_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'

# The number of images in the validation set.
_NUM_VALIDATION = 350

# Seed for repeatability.
_RANDOM_SEED = 0

# The number of shards per dataset split.
_NUM_SHARDS = 5

# The path where the Flowers dataset is
dataset_dir = './data/flowers'

### downlaod `flower` dataset

In [5]:
if not tf.gfile.Exists(dataset_dir):
  tf.gfile.MakeDirs(dataset_dir)

In [6]:
# downlaod and uncompress dataset.tar.gz
if not tf.gfile.Exists(dataset_dir + '/flower_photos'):
  dataset_utils.download_and_uncompress_tarball(_DATA_URL, dataset_dir)

>> Downloading flower_photos.tgz 99.2%

In [7]:
def _get_filenames_and_classes(dataset_dir):
  """Returns a list of filenames and inferred class names.

  Args:
    dataset_dir: A directory containing a set of subdirectories representing
      class names. Each subdirectory should contain PNG or JPG encoded images.

  Returns:
    A list of image file paths, relative to `dataset_dir` and the list of
    subdirectories, representing class names.
  """
  flower_root = os.path.join(dataset_dir, 'flower_photos')
  directories = []
  class_names = []
  for filename in os.listdir(flower_root):
    path = os.path.join(flower_root, filename)
    if os.path.isdir(path):
      directories.append(path)
      class_names.append(filename)

  photo_filenames = []
  for directory in directories:
    for filename in os.listdir(directory):
      path = os.path.join(directory, filename)
      photo_filenames.append(path)

  return photo_filenames, sorted(class_names)

In [8]:
photo_filenames, class_names = _get_filenames_and_classes(dataset_dir)

In [9]:
print('dataset size: {}'.format(len(photo_filenames)))
print('class_names : {}'.format(class_names))

dataset size: 3670
class_names : ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [11]:
type(photo_filenames)

list

In [12]:
photo_filenames

['./data/flowers\\flower_photos\\daisy\\100080576_f52e8ee070_n.jpg',
 './data/flowers\\flower_photos\\daisy\\10140303196_b88d3d6cec.jpg',
 './data/flowers\\flower_photos\\daisy\\10172379554_b296050f82_n.jpg',
 './data/flowers\\flower_photos\\daisy\\10172567486_2748826a8b.jpg',
 './data/flowers\\flower_photos\\daisy\\10172636503_21bededa75_n.jpg',
 './data/flowers\\flower_photos\\daisy\\102841525_bd6628ae3c.jpg',
 './data/flowers\\flower_photos\\daisy\\1031799732_e7f4008c03.jpg',
 './data/flowers\\flower_photos\\daisy\\10391248763_1d16681106_n.jpg',
 './data/flowers\\flower_photos\\daisy\\10437754174_22ec990b77_m.jpg',
 './data/flowers\\flower_photos\\daisy\\10437770546_8bb6f7bdd3_m.jpg',
 './data/flowers\\flower_photos\\daisy\\10437929963_bc13eebe0c.jpg',
 './data/flowers\\flower_photos\\daisy\\10466290366_cc72e33532.jpg',
 './data/flowers\\flower_photos\\daisy\\10466558316_a7198b87e2.jpg',
 './data/flowers\\flower_photos\\daisy\\10555749515_13a12a026e.jpg',
 './data/flowers\\flower_ph

In [16]:
for i in range(len(class_names)):
    print(i)

0
1
2
3
4


In [17]:
class_names_to_ids = dict(zip(class_names, range(len(class_names))))

In [18]:
# print class names to ids
for key, value in class_names_to_ids.items():
  print('class name: {} -- index: {}'.format(key, value))

class name: daisy -- index: 0
class name: dandelion -- index: 1
class name: tulips -- index: 4
class name: roses -- index: 2
class name: sunflowers -- index: 3


### Divide into train and test

In [19]:
random.seed(_RANDOM_SEED)
random.shuffle(photo_filenames)
training_filenames = photo_filenames[_NUM_VALIDATION:]
validation_filenames = photo_filenames[:_NUM_VALIDATION]

In [20]:
print('training dataset size: {}'.format(len(training_filenames)))
print('validation dataset size: {}'.format(len(validation_filenames)))

training dataset size: 3320
validation dataset size: 350


In [22]:
class ImageReader(object):
    """Helper class that provides TensorFlow image coding utilities."""

    def __init__(self):
        # Initializes function that decodes RGB JPEG data.
        self._decode_jpeg_data = tf.placeholder(dtype=tf.string)
        # Decode a JPEG-encoded image to a uint8 tensor
        # contents: A `Tensor` of type `string`. 0-D.  The JPEG-encoded image.
        self._decode_jpeg = tf.image.decode_jpeg(self._decode_jpeg_data, channels=3)

    def read_image_dims(self, sess, image_data):
        image = self.decode_jpeg(sess, image_data)
        return image.shape[0], image.shape[1] # image_height, image_width

    def decode_jpeg(self, sess, image_data):
        image = sess.run(self._decode_jpeg,
                         feed_dict={self._decode_jpeg_data: image_data})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

In [23]:
def _get_dataset_filename(dataset_dir, split_name, shard_id):
    # split_name = 'train', 'validation'
    # 통채로 저장하는게 아니라 shard 단위로 저장
    output_filename = 'flowers_%s_%05d-of-%05d.tfrecord' % (split_name, shard_id, _NUM_SHARDS)
    return os.path.join(dataset_dir, output_filename)

In [24]:
def int64_feature(values):
    """Returns a TF-Feature of int64s.

    Args:
    values: A scalar or list of values.

    Returns:
    A TF-Feature.
    """
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

In [25]:
def bytes_feature(values):
    """Returns a TF-Feature of bytes.

    Args:
    values: A string.

    Returns:
    A TF-Feature.
    """
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))

In [26]:
def float_feature(values):
    """Returns a TF-Feature of floats.

    Args:
    values: A scalar of list of values.

    Returns:
    A TF-Feature.
    """
    if not isinstance(values, (tuple, list)):
        values = [values]
    return tf.train.Feature(float_list=tf.train.FloatList(value=values))

In [27]:
def _convert_dataset(split_name, filenames, class_names_to_ids, dataset_dir):
  """Converts the given filenames to a TFRecord dataset.

  Args:
    split_name: The name of the dataset, either 'train' or 'validation'.
    filenames: A list of absolute paths to png or jpg images.
    class_names_to_ids: A dictionary from class names (strings) to ids
      (integers).
    dataset_dir: The directory where the converted datasets are stored.
  """
  assert split_name in ['train', 'validation']

  num_per_shard = int(math.ceil(len(filenames) / float(_NUM_SHARDS)))
  # num_per_shard = int( 3320 / 5.0 ) = 664 for training dataset
  # num_per_shard = int( 350 / 5.0 ) = 70 for validation dataset

  with tf.Graph().as_default():
    image_reader = ImageReader()

    with tf.Session('') as sess:

      for shard_id in range(_NUM_SHARDS):
        output_filename = _get_dataset_filename(
            dataset_dir, split_name, shard_id)
        # output_filename -> '../data/flowers/flowers_train_00000-of-00005.tfrecord'
        #                    '../data/flowers/flowers_train_00001-of-00005.tfrecord'
        #                                       ...
        #                    '../data/flowers/flowers_train_00004-of-00005.tfrecord'
        
        # step 1
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
          start_ndx = shard_id * num_per_shard
          end_ndx = min((shard_id+1) * num_per_shard, len(filenames))
          for i in range(start_ndx, end_ndx):
            sys.stdout.write('\r>> Converting image %d/%d shard %d' % (
                i+1, len(filenames), shard_id))
            sys.stdout.flush()

            # Read the filename:
            # step 2
            image_data = tf.gfile.FastGFile(filenames[i], 'rb').read()
            height, width = image_reader.read_image_dims(sess, image_data)

            class_name = os.path.basename(os.path.dirname(filenames[i]))
            class_id = class_names_to_ids[class_name]

            #example = dataset_utils.image_to_tfexample(
            #    image_data, b'jpg', height, width, class_id)
            # step 3
            features = tf.train.Features(feature={'image/encoded': bytes_feature(image_data),
                                                  'image/format': bytes_feature(b'jpg'),
                                                  'image/class/label': int64_feature(class_id),
                                                  'image/height': int64_feature(height),
                                                  'image/width': int64_feature(width),
                                                 })
            
            # step 4            
            example = tf.train.Example(features=features)
            
            # step 5
            tfrecord_writer.write(example.SerializeToString())

  sys.stdout.write('\n')
  sys.stdout.flush()

## `TFRecords` format 파일 만드는 방법

```python
# Step 1: create a writer to write tfrecord to that file
tfrecord_writer = tf.python_io.TFRecordWriter(output_filename)

# Step 2: get serialized data (binary values and shape of image)
# 한 개의 example(우리가 말하는 data 하나)을 만들기 위해 필요한 정보를 모은다.
image_data = tf.gfile.FastGFile(filenames[i], 'rb').read()
height, width = image_reader.read_image_dims(sess, image_data)
class_name = os.path.basename(os.path.dirname(filenames[i]))
class_id = class_names_to_ids[class_name]

# Step 3: create a tf.train.Features object
features = tf.train.Features(feature={'image/encoded': bytes_feature(image_data),
                                      'image/format': bytes_feature(image_format),
                                      'image/class/label': int64_feature(class_id),
                                      'image/height': int64_feature(height),
                                      'image/width': int64_feature(width),
                                     })

# Step 4: create a sample containing of features defined above
example = tf.train.Example(features=features)

# Step 5: write the sample to the tfrecord file
tfrecord_writer.write(sample.SerializeToString())
tfrecord_writer.close()
```

In [28]:
# First, convert the training and validation sets.
_convert_dataset('train', training_filenames, class_names_to_ids, dataset_dir)
_convert_dataset('validation', validation_filenames, class_names_to_ids, dataset_dir)

>> Converting image 3320/3320 shard 4
>> Converting image 350/350 shard 4


In [30]:
class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [29]:
# Finally, write the labels file:
labels_to_class_names = dict(zip(range(len(class_names)), class_names))
dataset_utils.write_label_file(labels_to_class_names, dataset_dir)

In [31]:
# print class names to ids
for key, value in labels_to_class_names.items():
  print('label index: {} -- class name: {}'.format(key, value))

label index: 0 -- class name: daisy
label index: 1 -- class name: dandelion
label index: 2 -- class name: roses
label index: 3 -- class name: sunflowers
label index: 4 -- class name: tulips


In [32]:
def _clean_up_temporary_files(dataset_dir):
  """Removes temporary files used to create the dataset.

  Args:
    dataset_dir: The directory where the temporary files are stored.
  """
  filename = _DATA_URL.split('/')[-1]
  filepath = os.path.join(dataset_dir, filename)
  tf.gfile.Remove(filepath)

  tmp_dir = os.path.join(dataset_dir, 'flower_photos')
  tf.gfile.DeleteRecursively(tmp_dir)

In [33]:
_clean_up_temporary_files(dataset_dir)
print('\nFinished converting the Flowers dataset!')

FailedPreconditionError: Failed to remove a directory: ./data/flowers\flower_photos; Directory not empty